### 🎯Objective

The objective of this notebook is to simulate real-world pricing scenarios and quantify their impact on demand and revenue.

This notebook:
- Simulates price increase and decrease scenarios
- Evaluates revenue sensitivity to pricing decisions
- Identifies safe vs risky pricing zones
- Supports data-driven pricing recommendations
This bridges price elasticity analysis with practical revenue decision-making, similar to how pricing teams operate in real businesses.

### Importing Libraries

In [ ]:
import pandas as pd #importing pandas for data manipulation
import numpy as np #importing numpy for mathematical calculations
from sklearn.linear_model import LinearRegression

- pandas → data manipulation

- numpy → numerical calculations

- LinearRegression → simple, interpretable demand model

We intentionally use a simple linear model here to keep pricing decisions transparent and explainable, which is preferred in many business contexts.

### Loading the dataset

In [ ]:
df= pd.read_csv("ecommerce_pricing_cleaned.csv")
df.head()

,product_id,category,base_price,competitor_price,promotion_flag,season,customer_segment,day_of_week,channel,current_price,units_sold
0,1102,Electronics,153.93,23.94,True,Fall,Regular,Tue,Web,111.99,5.0
1,1435,Home,190.17,250.84,False,Spring,Premium,Sat,Mobile App,190.17,179.0
2,1860,Electronics,166.57,257.71,False,Summer,Regular,Mon,Web,166.57,141.0
3,1270,Electronics,215.97,238.38,False,Winter,Regular,Mon,Web,215.97,164.0
4,1106,Fashion,102.55,255.81,True,Spring,Regular,Mon,Web,81.71,1238.0


- This dataset contains cleaned e-commerce transaction data, including:

Current product prices

Units sold

Category and promotion context

- This data has already been cleaned and validated in earlier steps.

### Creating a Baseline Price

In [ ]:
baseline_revenue= df['current_price']*df['units_sold']
df['baseline_revenue']= baseline_revenue
baseline_summary= baseline_revenue.sum()
baseline_summary

np.float64(320957025.0799999)

This is:

- What the business currently earns

- Without changing prices

- Without optimization

The baseline revenue (~$320.9M) serves as the reference point against which all pricing scenarios are evaluated.

### Retraining the Model

In [ ]:
X= df[['current_price']]
y= df['units_sold']
model= LinearRegression()
model.fit(X,y)
model.coef_, model.intercept_

(array([-6.62928888]), np.float64(1461.6793185883666))

Model Output

Price coefficient: -6.63

Intercept: 1461.68

Interpretation:
For every 1 unit increase in price, demand drops by 6.6 units on average.

This means:

- Demand is highly price elastic

- Customers respond strongly to price changes

### Define Pricing Scenarios
We simulate controlled price changes, not random ones.

Common retail scenarios:

−10% price (discount)

+5% price (small increase)

+10% price (aggressive increase)

In [ ]:
df['price_minus_10']= df['current_price']*0.9
df['price_plus_5']= df['current_price']*1.05
df['price_plus_10']= df['current_price']*1.10

### Predict Demand Under Pricing Scenarios

In [ ]:
df['demand_baseline']= model.predict(df[['current_price']])
df['demand_minus_10']= model.predict(df[['price_minus_10']].rename(columns={'price_minus_10': 'current_price'}))
df['demand_plus_5']= model.predict(df[['price_plus_5']].rename(columns={'price_plus_5': 'current_price'}))
df['demand_plus_10']= model.predict(df[['price_plus_10']].rename(columns={'price_plus_10': 'current_price'}))

- We reuse the same trained demand model and feed it different price inputs.

Renaming columns ensures:

- Feature names match training data

- Predictions remain valid and consistent

This simulates “what-if” pricing experiments without retraining models.

### Calculate Revenue Under Each Scenario

In [ ]:
df['revenue_baseline']= df['current_price']*df['demand_baseline']
df['revenue_minus_10']= df['price_minus_10']*df['demand_minus_10']
df['revenue_plus_5']= df['price_plus_5']*df['demand_plus_5']
df['revenue_plus_10']= df['price_plus_10']*df['demand_plus_10']

#### Saving the dataset for further analysis

In [ ]:
df.to_csv("ecommerce_pricing_featured.csv", index=False)

Revenue is computed as:

Revenue=Price×Predicted Demand

This allows us to evaluate both price and volume effects simultaneously, which is essential for pricing decisions.

### Compare Total Revenue

In [ ]:
scenario_summary= pd.DataFrame({
    'Scenario':['Baseline','-10% Price', '+5% Price','+10% Price'],
    'Total_Revenue':[df['revenue_baseline'].sum(),
                     df['revenue_minus_10'].sum(),
                     df['revenue_plus_5'].sum(),
                     df['revenue_plus_10'].sum()]})
scenario_summary.sort_values(by='Total_Revenue',ascending=False)

,Scenario,Total_Revenue
1,-10% Price,4.890323e+08
0,Baseline,3.209570e+08
2,+5% Price,2.202385e+08
3,+10% Price,1.083993e+08


This table summarizes the net business impact of each pricing strategy.
It answers the executive question:

“Which pricing decision maximizes total revenue?”

Key Insights

- The analysis shows that demand is highly price elastic.
- A 10% price reduction increases volume sufficiently to generate higher overall revenue than the current baseline.
- Conversely, price increases lead to disproportionate demand loss, making upward pricing risky without differentiation or demand insulation

Business Interpretation
- Leaving money on the table

- Pricing above revenue-optimal levels

What pricing teams should consider

- Strategic discounts (not blanket ones)

- Volume-driven growth strategies

- Inventory readiness before price cuts

These results represent a theoretical revenue optimum under unconstrained supply and should be combined with inventory and margin considerations before execution.

#### This business operates in a highly elastic pricing environment where revenue is maximized through volume expansion rather than price increases.